In [57]:
import pandas as pd

In [58]:
def map_income(income):
    if income<=9036.8:
        return 'Low'
    else:
        return 'High'

In [59]:
average_number = pd.read_csv('../../TABLAS LATEX/languages.csv', sep=';', decimal=',')
average_number = average_number[['Language', 'Average']]
average_number

,Language,Average
0,English,5607.00
1,Spanish,2348.33
2,French,832.33
3,Chinese,770.00
4,Italian,579.00
5,Russian,560.33
6,Portuguese,462.67
7,Arabic,418.67
8,Japanese,410.33
9,German,388.67


# Preply

In [60]:
preply = pd.read_csv('../data/results/final_dataframes/preply.csv', index_col=0)
preply = preply.drop_duplicates(subset=['user_name', 'language'])

In [61]:
preply['income_level'] = preply['income_level'].apply(map_income)

In [ ]:
preply.head()

In [63]:
preply.columns

Index(['language', 'position', 'retrieval_date', 'is_featured', 'user_name',
       'url', 'nationality_full', 'avg_rating', 'num_ratings', 'teaches',
       'subjects', 'speaks', 'lessons', 'price', 'price_currency',
       'avatar_url', 'nationality', 'clean_name', 'sanitized_name', 'gender',
       'probability', 'count', 'first_name', 'probability_male',
       'probability_female', 'gender_tuned', 'income_level', 'Code'],
      dtype='object')

In [64]:
from scipy.stats import ks_2samp
import numpy as np

def hypothesis_test(group1, group2, alpha=0.1):
    st, p_value = ks_2samp(group1, group2)
    if p_value<alpha:
        return st, p_value
    else:
        return st, p_value
    
def compute_aggregated_feature_top_k(df, top_k, language_col, aggregation_col, target_cols, group1, group2):
    count_group1 = 'count_{}'.format(group1)
    count_group2 = 'count_{}'.format(group2)
    variance_group1 = 'variance_{}'.format(group1)
    variance_group2 = 'variance_{}'.format(group2)
    mean_group1 = 'mean_{}'.format(group1)
    mean_group2 = 'mean_{}'.format(group2)
    median_group1 = 'median_{}'.format(group1)
    median_group2 = 'median_{}'.format(group2)
    
    results = pd.DataFrame(columns=['language', 'top_k', 'target_col', 'aggregation_col', mean_group1, mean_group2, median_group1, median_group2, count_group1, count_group2, variance_group1, variance_group2, 'statistic', 'p_value'])
    
    for lang in df[language_col].unique():
        temp = df[df[language_col]==lang]
        temp = temp.sort_values(by='position', ascending=True)
        
        for target in target_cols:
            temp = temp.dropna(subset=[target])
            if top_k is not None:
                temp = temp.head(top_k)
            
            temp[target] = pd.to_numeric(temp[target], errors='coerce')
            g1 = temp[temp[aggregation_col]==group1][target].values
            g2 = temp[temp[aggregation_col]==group2][target].values
            
            g1_count = len(g1)
            g2_count = len(g2)

            g1_mean = np.nanmean(g1) if g1_count else None
            g2_mean = np.nanmean(g2) if g2_count else None
            
            g1_median = np.nanmedian(g1) if g1_count else None
            g2_median = np.nanmedian(g2) if g2_count else None
            
            g1_var = g1.var() if g1_count else None
            g2_var = g2.var() if g2_count else None
            
            #Hypothesis testing
            if len(g1)>0 and len(g2)>0:
                st, p_value = hypothesis_test(g1, g2)
            else:
                st, p_value = None, None
            
            results = results.append({'language': lang, 'top_k': len(temp), 'target_col': target, 'aggregation_col': aggregation_col,
                                      mean_group1: g1_mean, mean_group2: g2_mean, median_group1: g1_median, median_group2: g2_median, count_group1: g1_count, count_group2: g2_count,
                                      variance_group1: g1_var, variance_group2: g2_var, 'statistic': st, 'p_value': p_value}, ignore_index=True)

    return results

In [65]:
preply_results_40 = compute_aggregated_feature_top_k(preply, 40, 'language', 'income_level', ['price', 'avg_rating', 'num_ratings'], 'High', 'Low')

<ipython-input-64-5b4dc0134a7f>:39: RuntimeWarning: Mean of empty slice
  g1_mean = np.nanmean(g1) if g1_count else None
/Users/javi/miniconda3/envs/tfg/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
<ipython-input-64-5b4dc0134a7f>:39: RuntimeWarning: Mean of empty slice
  g1_mean = np.nanmean(g1) if g1_count else None
/Users/javi/miniconda3/envs/tfg/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


In [66]:
preply_results_all = compute_aggregated_feature_top_k(preply, None, 'language', 'income_level', ['price', 'avg_rating', 'num_ratings'], 'High', 'Low')

<ipython-input-64-5b4dc0134a7f>:39: RuntimeWarning: Mean of empty slice
  g1_mean = np.nanmean(g1) if g1_count else None
/Users/javi/miniconda3/envs/tfg/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
<ipython-input-64-5b4dc0134a7f>:39: RuntimeWarning: Mean of empty slice
  g1_mean = np.nanmean(g1) if g1_count else None
/Users/javi/miniconda3/envs/tfg/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


In [67]:
preply_results = pd.concat([preply_results_40, preply_results_all])

In [68]:
preply_results = pd.merge(preply_results, average_number, how='left', left_on='language', right_on='Language')
preply_results = preply_results.rename(columns={'Average': 'average_num_teachers'})

In [69]:
preply_results.head()

,language,top_k,target_col,aggregation_col,mean_High,mean_Low,median_High,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
0,Thai,40,price,income_level,13,13.359,13,13,1,39,0,9.76857,0.384615,1,Thai,51.67
1,Thai,40,avg_rating,income_level,5,4.93793,5,5,1,39,0,NaN,0.25641,1,Thai,51.67
2,Thai,40,num_ratings,income_level,1,7.62069,1,5,1,39,0,NaN,0.897436,0.25,Thai,51.67
3,German,40,price,income_level,29.6667,27,29,27,39,1,93.9658,0,0.564103,0.9,German,388.67
4,German,40,avg_rating,income_level,4.95,4.9,5,4.9,39,1,NaN,0,0.717949,0.6,German,388.67


In [100]:
preply_results.to_csv('../data/results/features_analysis/income/preply.csv', sep=',')

In [71]:
preply_results[preply_results['p_value']<0.1]

,language,top_k,target_col,aggregation_col,mean_High,mean_Low,median_High,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
6,Portuguese,40,price,income_level,16.625,14.375,17,13,16,24,11.7344,67.2344,0.458333,0.0271981,Portuguese,462.67
11,Hebrew,40,num_ratings,income_level,5.6,19,3.5,19,37,3,NaN,NaN,0.702703,0.0736842,Hebrew,46.67
32,Russian,40,num_ratings,income_level,9.28571,40,6.5,17,30,10,NaN,NaN,0.566667,0.0115179,Russian,560.33
36,English,40,price,income_level,14.6333,9.3,13,9.5,30,10,13.5656,1.41,0.8,3.22299e-05,English,5607.00
38,English,31,num_ratings,income_level,13.875,17.2857,3.5,13,24,7,685.693,166.204,0.607143,0.0218781,English,5607.00
49,Vietnamese,36,avg_rating,income_level,NaN,4.95263,NaN,5,5,31,NaN,NaN,0.612903,0.0455288,Vietnamese,49.67
50,Vietnamese,36,num_ratings,income_level,NaN,7,NaN,3,5,31,NaN,NaN,0.612903,0.0455288,Vietnamese,49.67
59,Spanish,35,num_ratings,income_level,11.1515,23.5,8,23.5,33,2,106.916,2.25,0.848485,0.0705882,Spanish,2348.33
60,French,40,price,income_level,25.4375,15.5,24,15,32,8,66.2461,20.5,0.6875,0.00236807,French,832.33
69,German,462,price,income_level,23.187,16.086,21,15,369,93,74.5694,61.0249,0.389632,1.29504e-10,German,388.67


# Italki

In [72]:
italki = pd.read_csv('../data/results/final_dataframes/italki.csv', index_col=0)
italki = italki.drop_duplicates(subset=['user_id', 'language'])

In [73]:
italki['income_level'] = italki['income_level'].apply(map_income)

In [ ]:
italki.head()

In [75]:
italki.columns

Index(['position', 'retrieval_date', 'user_id', 'user_name',
       'avatar_file_name', 'video_picture', 'is_pro', 'nationality', 'teaches',
       'also_speaks', 'in_platform_since', 'rating', 'number_sessions',
       'price', 'price_time', 'price_currency', 'clean_name', 'sanitized_name',
       'gender', 'probability', 'count', 'language', 'probability_male',
       'probability_female', 'gender_tuned', 'income_level', 'Code'],
      dtype='object')

In [76]:
italki_results_40 = compute_aggregated_feature_top_k(italki, 40, 'language', 'income_level', ['price', 'rating', 'number_sessions'], 'High', 'Low')

In [77]:
italki_results_all = compute_aggregated_feature_top_k(italki, None, 'language', 'income_level', ['price', 'rating', 'number_sessions'], 'High', 'Low')

In [78]:
italki_results = pd.concat([italki_results_40, italki_results_all])

In [79]:
italki_results = pd.merge(italki_results, average_number, how='left', left_on='language', right_on='Language')
italki_results = italki_results.rename(columns={'Average': 'average_num_teachers'})

In [80]:
italki_results.head()

,language,top_k,target_col,aggregation_col,mean_High,mean_Low,median_High,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
0,Thai,40,price,income_level,1800,1017.92,1800,1000,1,39,0,79232.6,1,0.05,Thai,51.67
1,Thai,40,rating,income_level,5,4.85641,5,5,1,39,0,0.622459,0.153846,1,Thai,51.67
2,Thai,40,number_sessions,income_level,511,329.385,511,106,1,39,0,200154,0.794872,0.45,Thai,51.67
3,German,40,price,income_level,2562.61,1005.5,2000,900,36,4,2.10949e+06,293491,0.694444,0.035343,German,388.67
4,German,40,rating,income_level,4.84722,5,5,5,36,4,0.672492,0,0.166667,0.999825,German,388.67


In [98]:
italki_results.to_csv('../data/results/features_analysis/income/italki.csv', sep=',')

In [82]:
italki_results[italki_results['p_value']<0.1]

,language,top_k,target_col,aggregation_col,mean_High,mean_Low,median_High,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
0,Thai,40,price,income_level,1800,1017.92,1800,1000,1,39,0,79232.6,1,0.05,Thai,51.67
3,German,40,price,income_level,2562.61,1005.5,2000,900,36,4,2.10949e+06,293491,0.694444,0.035343,German,388.67
38,English,40,number_sessions,income_level,1086.5,1750.67,256,960,34,6,3.339e+06,3.15583e+06,0.588235,0.0361871,English,5607.00
57,French,40,price,income_level,1781.81,887.5,1900,900,36,4,445820,15468.8,0.805556,0.0072218,French,832.33
63,Persian (Farsi),40,price,income_level,1450,913.632,1450,800,2,38,22500,175122,0.894737,0.0384615,NaN,NaN
66,Thai,79,price,income_level,1650,1061.88,1650,1000,2,77,22500,90318.5,0.87013,0.0428432,Thai,51.67
69,German,400,price,income_level,2080.51,1314.3,1800,1200,367,33,984280,269898,0.431343,1.25281e-05,German,388.67
72,Portuguese,352,price,income_level,1574.85,1027.07,1400,975,75,277,981628,209783,0.401252,4.68856e-09,Portuguese,462.67
74,Portuguese,352,number_sessions,income_level,888.613,613.354,548,245,75,277,1.45058e+06,1.19085e+06,0.269314,0.000283293,Portuguese,462.67
75,Hebrew,65,price,income_level,1868.73,1000,1600,1000,63,2,788240,40000,0.809524,0.0875,Hebrew,46.67


# Verbling

In [83]:
verbling = pd.read_csv('../data/results/final_dataframes/verbling.csv', index_col=0)
verbling = verbling.drop_duplicates(subset=['first_name', 'last_name', 'language'])

In [84]:
verbling['income_level'] = verbling['income_level'].apply(map_income)

In [ ]:
verbling.head()

In [86]:
verbling.columns

Index(['language', 'position', 'retrieval_date', 'is_featured', 'first_name',
       'last_name', 'url', 'nationality', 'location', 'avg_rating',
       'avg_lessons_per_students', 'num_ratings', 'teaching_levels', 'teaches',
       'class_details', 'speaks', 'lessons', 'students', 'dialect',
       'price_currency', 'avatar_url', 'clean_name', 'sanitized_name',
       'gender', 'probability', 'count', 'price_detail', 'price',
       'probability_male', 'probability_female', 'gender_tuned',
       'income_level', 'Code'],
      dtype='object')

In [87]:
verbling_results_40 = compute_aggregated_feature_top_k(verbling, 40, 'language', 'income_level', ['price', 'avg_rating', 'num_ratings', 'avg_lessons_per_students'], 'High', 'Low')

In [88]:
verbling_results_all = compute_aggregated_feature_top_k(verbling, None, 'language', 'income_level', ['price', 'avg_rating', 'num_ratings', 'avg_lessons_per_students'], 'High', 'Low')

In [89]:
verbling_results = pd.concat([verbling_results_40, verbling_results_all])

In [90]:
verbling_results = pd.merge(verbling_results, average_number, how='left', left_on='language', right_on='Language')
verbling_results = verbling_results.rename(columns={'Average': 'average_num_teachers'})

In [91]:
verbling_results.head()

,language,top_k,target_col,aggregation_col,mean_High,mean_Low,median_High,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
0,Thai,18,price,income_level,21,17.4667,20,17,3,15,2,7.04889,0.8,0.0490196,Thai,51.67
1,Thai,18,avg_rating,income_level,3.32432,4.65972,4.97297,5,3,15,5.52569,1.55121,0.533333,0.411765,Thai,51.67
2,Thai,18,num_ratings,income_level,88.6667,58.3333,2,10,3,15,15371.6,5577.16,0.466667,0.571078,Thai,51.67
3,Thai,18,avg_lessons_per_students,income_level,16.5667,14.3333,23,12.2,3,15,139.509,100.5,0.4,0.735294,Thai,51.67
4,German,40,price,income_level,32.4642,None,30,None,40,0,73.5254,None,None,None,German,388.67


In [99]:
verbling_results.to_csv('../data/results/features_analysis/income/verbling.csv', sep=',')

In [93]:
verbling_results[verbling_results['p_value']<0.1]

,language,top_k,target_col,aggregation_col,mean_High,mean_Low,median_High,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
0,Thai,18,price,income_level,21,17.4667,20,17,3,15,2,7.04889,0.8,0.0490196,Thai,51.67
8,Portuguese,40,price,income_level,20.2857,14.1209,18,13,7,33,21.0612,19.6854,0.666667,0.00582829,Portuguese,462.67
43,Russian,40,avg_lessons_per_students,income_level,15.8187,28.4875,14.55,19.6,32,8,67.2059,364.199,0.625,0.00830405,Russian,560.33
48,English,40,price,income_level,24.4211,13.5,23.15,13.5,38,2,55.1711,2.25,0.921053,0.025641,English,5607.00
51,English,40,avg_lessons_per_students,income_level,15.6526,10.35,15.05,10.35,38,2,27.6914,0.0625,0.868421,0.0538462,English,5607.00
81,French,40,avg_rating,income_level,4.84813,4.9586,5,4.9586,38,2,0.63729,0.000143628,0.842105,0.0717949,French,832.33
88,Thai,18,price,income_level,21,17.4667,20,17,3,15,2,7.04889,0.8,0.0490196,Thai,51.67
96,Portuguese,89,price,income_level,19.5789,14.9684,20,15,19,70,13.2964,29.2353,0.547368,0.000106242,Portuguese,462.67
131,Russian,89,avg_lessons_per_students,income_level,16.3932,22.8563,13.9,17.55,73,16,170.76,310.407,0.35274,0.057546,Russian,560.33
136,English,424,price,income_level,22.8948,15.1491,21,14.5,367,57,67.8065,26.7453,0.548831,2.28706e-14,English,5607.00
